In [10]:
import numpy as np
from keras.models import load_model
import imgaug as ia
from imgaug import augmenters as iaa

# Define Paths and load data

In [3]:
train_img_path = '../data/train/'
model_name = 'ksamxamx.h5'
model_path = '../models/' + model_name
x_train_path = train_img_path + 'x_train.npy'
y_train_path = train_img_path + 'y_train.npy'
x_valid_path = train_img_path + 'x_valid.npy'
y_valid_path = train_img_path + 'y_valid.npy'


x_train = (train_img_path + x_train_path)
y_train= (train_img_path + y_train_path)
x_valid = (train_img_path + x_valid_path)
y_valid = (train_img_path + y_valid_path)

# Define augmentation sequence

In [11]:
# sometimes = lambda aug: iaa.Sometimes(0.5, aug)
# augment 50% of the time images: flip left right || affine || crop and pad
aug_sequence = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.OneOf([
        iaa.Noop(),
        iaa.Affine(rotate=(-10, 10), translate_percent={"x": (-0.25, 0.25)}, mode='symmetric', cval=(0), backend="cv2"),
        iaa.Noop(),
        iaa.CropAndPad(
            percent=(-0.2, 0.2),
            pad_mode="reflect",
            pad_cval=0,
            keep_size=False
        ),
    ])
])


# method for image augmentation
def augment(x_train, y_train):
    seq_det = aug_sequence.to_deterministic()
    X_train_aug = seq_det.augment_image(x_train)
    
    y_train_aug = seq_det.augment_image(y_train)

    return np.array(x_train_aug), np.array(y_train_aug)


# Define image generator with augmentation

In [12]:
# data generator from given images
def make_image_gen(x_train, y_train, batch_size=32):
    all_batches_index = np.arange(0, x_train.shape[0])
    out_images = []
    out_masks = []
    
    while True:
        np.random.shuffle(all_batches_index)
        for index in all_batches_index:
            img, mask = augment(x_train[index], y_train[index])

            out_images += [img]
            out_masks += [mask]
            if len(out_images) >= batch_size:
                yield np.stack(out_images, 0), np.stack(out_masks, 0)
                out_images, out_masks = [], []